# W&B Example - LLM

### Setting up OpenAI API KEY

In [ ]:
import os
os.environ['OPENAI_API_KEY'] = "sk-xxx"

### Enabling W&B tracing

In [ ]:
os.environ["LANGCHAIN_WANDB_TRACING"] = "true"
os.environ["WANDB_PROJECT"] = "product_description_test"

### Langchain model and prompts

In [ ]:
# Prompt Engineering
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import ChatPromptTemplate
from langchain.chains import SequentialChain

llm = ChatOpenAI(temperature=0.2, model="gpt-3.5-turbo")

keywords_prompt_template = ChatPromptTemplate.from_messages([
    ("system", """
    You are a helpful assistant that generate a CSV list of keywords related to a product summary

    Example Format:
PRODUCT NAME: product name here
KEYWORDS: keywords separated by commas here

Generate five to ten keywords that would increase product visibility. Begin!

    """),
    ("human", """
PRODUCT NAME: {product_name}
KEYWORDS:""")
])
keywords_chain = LLMChain(llm=llm, prompt=keywords_prompt_template, output_key="keywords")

product_prompt_template = ChatPromptTemplate.from_messages([
    ("system", """As a Product Description Generator, generate a multi paragraph rich text product description with emojis based on the information provided in the product name and keywords separated by commas.

Example Format:
PRODUCT NAME: product name here
KEYWORDS: keywords separated by commas here
PRODUCT DESCRIPTION: product description here

Generate a product description that is creative and SEO compliant. Emojis should be added to make product description look appealing. Begin!

"""),
    ("human", """
PRODUCT NAME: {product_name}
KEYWORDS: {keywords}
PRODUCT DESCRIPTION:
    """)
])

product_chain = LLMChain(
    prompt=product_prompt_template,
    llm=llm,
    output_key="description"
)

overall_chain = SequentialChain(
    chains=[keywords_chain, product_chain],
    input_variables=["product_name"],
    # Here we return multiple variables
    output_variables=["description"])

### Giskard model and dataset

In [ ]:
import giskard
import pandas as pd

dataset = giskard.Dataset(pd.DataFrame({
    'product_name': ["Double cats bowl",
                     "Automatic food dispenser for pets",
                     "Small Tennis ball"],

}), name="Test dataset",
    column_types={"product_name": "text"})

model = giskard.Model(overall_chain, name="Product keywords and description generator", model_type="text_generation",
                      description="Generate product description based on a product's name and the associated keywords. "
                                  "Description should be using emojis and being SEO compliant.",
                      feature_names=['product_name'])

### Examples

In [ ]:
predictions = model.predict(dataset).prediction
for k, v in dataset.df.product_name.to_dict().items():
  print("Example #", k+1)
  print("product_name (input):", v)
  print("product_description (output):", predictions[k])
  print("--------------------------------------------------------------------")

### Giskard scan

In [ ]:
# Generate scan report
report = giskard.scan(model, dataset, raise_exceptions=True)

# Generate test suite from scan report
test_suite = report.generate_test_suite()

In [ ]:
pd.set_option("display.max_colwidth", 999)
display(report)

### Logging results to W&B

In [ ]:
import wandb
wandb.login(key="key to retrieve from https://wandb.ai/authorize")
run = wandb.init(project=os.environ["WANDB_PROJECT"])

# Logging the scan report
report.to_wandb(run)

# Logging the test suite generated by the scan
test_suite.run().to_wandb(run)
run.finish()